# Oracle DB Access
Basic access to an Oracle Database with JDBC,  
using a url like `jdbc:oracle:thin:@//100.111.136.104:1521/BOTS.localdomain`.

### Using Maven to get to the JDBC driver

In [1]:
// Proxy settings
System.out.println(String.format("HTTP Proxy: %s", System.getProperty("HTTP_PROXY")));
System.out.println(String.format("HTTPS Proxy: %s", System.getProperty("HTTPS_PROXY")));

HTTP Proxy: null
HTTPS Proxy: null


In [2]:
// %maven com.oracle:ojdbc14:10.2.0.2.0
%maven com.oracle.database.jdbc:ojdbc10:19.12.0.0
// %maven com.oracle.jdbc:ojdbc8:19.3.0.0.9

In [3]:
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.Driver;
import java.sql.PreparedStatement;
import java.sql.ResultSet;
import java.sql.ResultSetMetaData;
import java.sql.Statement;
import java.util.stream.Collectors;

In [4]:
final static String JDBC_HOSTNAME = "100.111.136.104";
final static int JDBC_PORT = 1521;
final static String JDBC_SERVICE_NAME = "BOTS.localdomain";

final static String USERNAME = "races";
final static String PASSWORD = "racesracesracesraces";

In [5]:
String jdbcUrl = String.format("jdbc:oracle:thin:@//%s:%d/%s",
                                JDBC_HOSTNAME,
                                JDBC_PORT,
                                JDBC_SERVICE_NAME);

try {
    Class.forName("oracle.jdbc.driver.OracleDriver");
    System.out.println(">> Driver loaded");
    Enumeration<Driver> drivers = DriverManager.getDrivers();
    while (drivers.hasMoreElements()) {
        Driver driver = drivers.nextElement();
        System.out.println(String.format("Version %d.%d", driver.getMajorVersion(), driver.getMinorVersion()));
    }
} catch (ClassNotFoundException cnfe) {
    cnfe.printStackTrace();
}

System.out.printf("Connecting with [%s]\n", jdbcUrl);
Connection connection = DriverManager.getConnection(jdbcUrl, USERNAME, PASSWORD);
System.out.println(">> Connected");


>> Driver loaded
Version 19.12
Connecting with [jdbc:oracle:thin:@//100.111.136.104:1521/BOTS.localdomain]
>> Connected


In [6]:
String sqlStmt = "SELECT * FROM RACE";

In [7]:
ResultSet rs;
PreparedStatement preparedStatement = null;
Statement statement = null;

In [8]:
statement = connection.createStatement();
rs = statement.executeQuery(sqlStmt);

In [9]:
ResultSetMetaData metaData = rs.getMetaData();
int columnCount = metaData.getColumnCount();
System.out.println(String.format("Returned %d columns", columnCount));

Returned 5 columns


In [10]:
int nbRows = 0;
while (rs.next()) {
    nbRows++;
    for (int i = 0; i < columnCount; i++) {
        String colName = metaData.getColumnName(i + 1);
        Object colValue = rs.getObject(i+1);
        System.out.println(String.format("Row #%d, %s: %s", nbRows, colName, colValue));
    }
}

Row #1, RACE_ID: 1
Row #1, NAME: Rolex 24 At Daytona
Row #1, RACE_CLASS: DP/GT
Row #1, RACE_DATE: January 26 January 27
Row #1, TRACK_ID: 1
Row #2, RACE_ID: 2
Row #2, NAME: Gainsco Grand Prix of Miami
Row #2, RACE_CLASS: DP/GT
Row #2, RACE_DATE: March 29
Row #2, TRACK_ID: 2
Row #3, RACE_ID: 3
Row #3, NAME: Mexico City 250
Row #3, RACE_CLASS: DP/GT
Row #3, RACE_DATE: April 19
Row #3, TRACK_ID: 2
Row #4, RACE_ID: 4
Row #4, NAME: Bosch Engineering 250 at VIR
Row #4, RACE_CLASS: GT
Row #4, RACE_DATE: April 27
Row #4, TRACK_ID: 4
Row #5, RACE_ID: 5
Row #5, NAME: RumBum.com 250
Row #5, RACE_CLASS: DP/GT
Row #5, RACE_DATE: May 17
Row #5, TRACK_ID: 5
Row #6, RACE_ID: 6
Row #6, NAME: Lime Rock GT Classic 250
Row #6, RACE_CLASS: GT
Row #6, RACE_DATE: May 26
Row #6, TRACK_ID: 6
Row #7, RACE_ID: 7
Row #7, NAME: Sahlen's Six Hours of the Glen
Row #7, RACE_CLASS: DP/GT
Row #7, RACE_DATE: June 7
Row #7, TRACK_ID: 7


Done with the query, closing resources.  
**Note**:  
Thnis could be done with a `try with resources`, this is done explicitely here for clarity.

In [11]:
rs.close();
statement.close();

Using a `try with resources`, as an example.

In [12]:
try (Statement statement = connection.createStatement()) {
    try (ResultSet resultSet = statement.executeQuery("SELECT * FROM RACE")) {

        ResultSetMetaData metaData = resultSet.getMetaData();
        int columnCount = metaData.getColumnCount();

        while (resultSet.next()) {

            List<Object> oneRow = new ArrayList<>();
            for (int i = 0; i < columnCount; i++) {
                oneRow.add(resultSet.getObject(i + 1));
            }
            System.out.println(oneRow.stream().map(obj -> String.valueOf(obj)).collect(Collectors.joining(", ")));
        }
    }
}

1, Rolex 24 At Daytona, DP/GT, January 26 January 27, 1
2, Gainsco Grand Prix of Miami, DP/GT, March 29, 2
3, Mexico City 250, DP/GT, April 19, 2
4, Bosch Engineering 250 at VIR, GT, April 27, 4
5, RumBum.com 250, DP/GT, May 17, 5
6, Lime Rock GT Classic 250, GT, May 26, 6
7, Sahlen's Six Hours of the Glen, DP/GT, June 7, 7


In [13]:
connection.close();
System.out.println("Bye");

Bye


---